In [2]:
import json
import folium

import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from datetime import time, timedelta, datetime

In [3]:
def find_time_window(route_data, package_data, key, stop_id):
    
    """Inputs: Key: route_id
                Stop_id: 2 alphabet id for stop"""
    
    route_start = datetime.fromisoformat(
        route_data[k]['date_YYYY_MM_DD'] + ' ' + route_data[k]['departure_time_utc'])
    packages = package_data[key][stop_id]
    end_stop = datetime.max
    start_stop = route_start
    delta_max = end_stop - start_stop
    for pid, pinfo in packages.items():
        if isinstance(pinfo['time_window']['end_time_utc'], str):
            end = datetime.fromisoformat(pinfo['time_window']['end_time_utc'])
            start = datetime.fromisoformat(pinfo['time_window']['start_time_utc'])
            end_stop = min(end, end_stop)
            start_stop = max(start, start_stop)
    delta = (end_stop - start_stop)
    if delta < delta_max:
        return ((start_stop - route_start).total_seconds(), (end_stop - route_start).total_seconds())
    else:
        return None
    
def find_service_time(package_data, key, stop_id):
    t = 0
    for p, pinfo in package_data[key][stop_id].items():
        t = t + pinfo['planned_service_time_seconds']
    return t

In [4]:
with open('/Users/ashutoshshukla/Desktop/Current_Projects/MITRC/version_1/data/model_build_inputs/route_data.json') as f:
    data = json.load(f)
    
with open('/Users/ashutoshshukla/Desktop/Current_Projects/MITRC/version_1/data/model_build_inputs/actual_sequences.json') as f:
    data_actual = json.load(f)
    
with open('/Users/ashutoshshukla/Desktop/Current_Projects/MITRC/version_1/data/model_build_inputs/travel_times.json') as f:
    data_travel_time = json.load(f)
    
with open('/Users/ashutoshshukla/Desktop/Current_Projects/MITRC/version_1/data/model_build_inputs/package_data.json') as f:
    data_package = json.load(f)

In [9]:
route_ids = list(data.keys())
route_data = data.copy()
package_data = data_package.copy()

time_dictionary = {}

counter = 0
for k in package_data:
    counter = counter + 1
    time_dictionary[k] = {}
    for s in package_data[k]:
        time_dictionary[k][s] = {}
        time_dictionary[k][s]["time_window"] = find_time_window(route_data, package_data, k, s)
        time_dictionary[k][s]["service_window"] = find_service_time(package_data, k, s)

## Input for routing engine

In [18]:
route_key = 'RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77'

distance_matrix = pd.DataFrame(data_travel_time[route_key])

time_window_list = []

for i in time_dictionary[route_key]:    
    none_check = time_dictionary[route_key][i]['time_window']
    if none_check is None:
        time_window_list.append((0,28670))
    else:
        time_window_list.append((int(none_check[0]),int(none_check[1])))

In [19]:
station_index = 0

for j in route_data[route_key]['stops'].keys():
    dropoff_type = route_data[route_key]['stops'][j]['type']
    if dropoff_type == 'Station':
        break
    else:
        station_index = station_index + 1

In [22]:
"""Vehicles Routing Problem (VRP) with Time Windows."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = distance_matrix.values
    data['time_windows'] = time_window_list
    data['num_vehicles'] = 1
    data['depot'] = station_index
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    
    solution_collector = []
    
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            temp = manager.IndexToNode(index)
            solution_collector.append(temp)
            plan_output += '{0} Time({1},{2}) -> '.format(
                manager.IndexToNode(index), solution.Min(time_var),
                solution.Max(time_var))
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        print(plan_output)
        total_time += solution.Min(time_var)
    print('Total time of all routes: {}min'.format(total_time))
    return solution_collector

In [39]:
"""Solve the VRP with time windows."""

# Instantiate the data problem.
data = create_data_model()

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']), data['num_vehicles'], data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)


# Create and register a transit callback.
def time_callback(from_index, to_index):
    """Returns the travel time between the two nodes."""
    # Convert from routing variable Index to time matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['time_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(time_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Time Windows constraint.
time = 'Time'
routing.AddDimension(
    transit_callback_index,
    50000,  # allow waiting time
    50000,  # maximum time per vehicle
    False,  # Don't force start cumul to zero.
    time)

time_dimension = routing.GetDimensionOrDie(time)

# Add time window constraints for each location except depot.
for location_idx, time_window in enumerate(data['time_windows']):
    if location_idx == data['depot']:
        continue
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(int(time_window[0]), int(time_window[1]))
    
    
# Add time window constraints for each vehicle start node.
depot_idx = data['depot']
for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    time_dimension.CumulVar(index).SetRange(
        data['time_windows'][depot_idx][0],
        data['time_windows'][depot_idx][1])

    
# Instantiate route start and end times to produce feasible times.
for i in range(data['num_vehicles']):
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.Start(i)))
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.End(i)))

    
# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    solution_collector = print_solution(data, manager, routing, solution)

Route for vehicle 0:
103 Time(0,0) -> 89 Time(1651,3533) -> 31 Time(1668,3550) -> 40 Time(1703,3585) -> 2 Time(1792,3674) -> 77 Time(1857,3739) -> 8 Time(1931,3813) -> 85 Time(1981,3863) -> 47 Time(2014,3896) -> 104 Time(2046,3928) -> 16 Time(2072,3954) -> 69 Time(2138,4020) -> 53 Time(2202,4084) -> 74 Time(2249,4131) -> 70 Time(2303,4185) -> 32 Time(2309,4191) -> 81 Time(2343,4225) -> 0 Time(2360,4242) -> 24 Time(2384,4266) -> 93 Time(2402,4284) -> 117 Time(2479,4361) -> 41 Time(2630,4512) -> 48 Time(2682,4564) -> 92 Time(2835,4717) -> 112 Time(2987,4869) -> 62 Time(3009,4891) -> 36 Time(3028,4910) -> 26 Time(3037,4919) -> 45 Time(3074,4956) -> 86 Time(3093,4975) -> 13 Time(3115,4997) -> 35 Time(3205,5087) -> 108 Time(3285,5167) -> 87 Time(3319,5201) -> 111 Time(3370,5252) -> 96 Time(3387,5269) -> 78 Time(3404,5286) -> 21 Time(3415,5297) -> 105 Time(3453,5335) -> 39 Time(3500,5382) -> 71 Time(3513,5395) -> 4 Time(3536,5418) -> 115 Time(3579,5461) -> 95 Time(3683,5565) -> 19 Time(3803,

In [25]:
index_set_of_nodes = list(pd.DataFrame(data_travel_time[route_key]).index)

In [40]:
for i in range(len(time_window_list)):
    print(i, time_window_list[i])

0 (0, 28670)
1 (0, 28670)
2 (0, 28670)
3 (0, 28670)
4 (0, 28670)
5 (0, 25070)
6 (5270, 32270)
7 (0, 28670)
8 (0, 28670)
9 (0, 28670)
10 (0, 25070)
11 (0, 25070)
12 (0, 25070)
13 (0, 28670)
14 (0, 28670)
15 (0, 28670)
16 (0, 28670)
17 (0, 28670)
18 (0, 28670)
19 (0, 14270)
20 (0, 25070)
21 (0, 28670)
22 (0, 28670)
23 (0, 25070)
24 (0, 28670)
25 (0, 28670)
26 (0, 28670)
27 (0, 28670)
28 (0, 28670)
29 (0, 28670)
30 (0, 28670)
31 (0, 28670)
32 (0, 28670)
33 (0, 28670)
34 (0, 28670)
35 (0, 28670)
36 (0, 28670)
37 (0, 25070)
38 (0, 28670)
39 (0, 28670)
40 (0, 28670)
41 (0, 28670)
42 (0, 25070)
43 (0, 28670)
44 (0, 28670)
45 (0, 28670)
46 (0, 28670)
47 (0, 28670)
48 (0, 28670)
49 (0, 28670)
50 (0, 25070)
51 (0, 28670)
52 (0, 28670)
53 (0, 28670)
54 (0, 28670)
55 (0, 28670)
56 (0, 28670)
57 (0, 32270)
58 (0, 25070)
59 (0, 28670)
60 (0, 28670)
61 (0, 14270)
62 (0, 28670)
63 (0, 28670)
64 (0, 28670)
65 (0, 25070)
66 (7070, 28670)
67 (0, 28670)
68 (0, 28670)
69 (0, 28670)
70 (0, 28670)
71 (0, 286

In [27]:
temp = {}

for i in range(len(index_set_of_nodes)):
    temp[index_set_of_nodes[solution_collector[i]]] = i

In [38]:
temp

{'VE': 0,
 'TG': 1,
 'GP': 2,
 'HT': 3,
 'AG': 4,
 'QM': 5,
 'BY': 6,
 'SF': 7,
 'JH': 8,
 'VW': 9,
 'CW': 10,
 'NR': 11,
 'KN': 12,
 'PT': 13,
 'NU': 14,
 'GS': 15,
 'RG': 16,
 'AD': 17,
 'EX': 18,
 'TY': 19,
 'ZP': 20,
 'HW': 21,
 'JM': 22,
 'TQ': 23,
 'YN': 24,
 'MO': 25,
 'HG': 26,
 'FF': 27,
 'IP': 28,
 'SI': 29,
 'CM': 30,
 'HB': 31,
 'XD': 32,
 'SQ': 33,
 'YJ': 34,
 'UN': 35,
 'QO': 36,
 'EC': 37,
 'WJ': 38,
 'HR': 39,
 'PB': 40,
 'BE': 41,
 'ZB': 42,
 'UJ': 43,
 'DN': 44,
 'CP': 45,
 'CG': 46,
 'LG': 47,
 'ZE': 48,
 'UR': 49,
 'DJ': 50,
 'NL': 51,
 'VA': 52,
 'HN': 53,
 'TH': 54,
 'IW': 55,
 'GU': 56,
 'IJ': 57,
 'SC': 58,
 'XB': 59,
 'KP': 60,
 'MR': 61,
 'UW': 62,
 'BA': 63,
 'YH': 64,
 'DQ': 65,
 'LD': 66,
 'NE': 67,
 'KJ': 68,
 'GN': 69,
 'CA': 70,
 'TC': 71,
 'BT': 72,
 'KA': 73,
 'AF': 74,
 'KM': 75,
 'MW': 76,
 'CO': 77,
 'BG': 78,
 'KG': 79,
 'PX': 80,
 'SD': 81,
 'UU': 82,
 'YY': 83,
 'UI': 84,
 'QX': 85,
 'EO': 86,
 'IA': 87,
 'YE': 88,
 'KU': 89,
 'ZU': 90,
 'VC': 91

In [34]:
temp

route_number =  0
coordinates = []

for i in temp:
    lat = route_data[route_ids[route_number]]['stops'][i]['lat']
    long = route_data[route_ids[route_number]]['stops'][i]['lng']
    coordinates.append([lat, long])
    
# Create the map and add the line
m = folium.Map(location=[coordinates[0][0], coordinates[0][1]], zoom_start=12)
my_PolyLine=folium.PolyLine(locations=coordinates,weight=3)
m.add_child(my_PolyLine)
m

In [ ]:
with open('/Users/ashutoshshukla/Desktop/Current_Projects/MITRC/version_1/data/model_apply_inputs/new_route_data.json') as f:
    data = json.load(f)